In [10]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import pulp
from pulp.apis.glpk_api import GLPK_CMD
os.path.abspath(os.getcwd())

'd:\\GIT_REPOS\\2021-NCKU_ORA\\ASGMT_04'

# Data Envelopment Analysis Run Thru. Handout Examples

In [11]:
data = [['A', 1, 1], ['B', 2, 4], ['C', 4, 6], ['D', 6, 7], [
    'E', 9, 8], ['F', 5, 3], ['G', 4, 1], ['H', 10, 7], ['I', 8, 4]]
df = pd.DataFrame(data, columns=['DMU', 'Input', 'Output'])
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DMU     9 non-null      object
 1   Input   9 non-null      int64 
 2   Output  9 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 344.0+ bytes


,DMU,Input,Output
0,A,1,1
1,B,2,4
2,C,4,6
3,D,6,7
4,E,9,8
5,F,5,3
6,G,4,1
7,H,10,7
8,I,8,4


In [12]:
X = [df.columns.tolist()[1]]
Y = [df.columns.tolist()[2]]
print(X)
print(Y)

['Input']
['Output']


###  Overall Efficiency (OE, CRS Primal model)

In [13]:
OE_crs = []
U_crs = []
V_crs = []
for r in range(df.shape[0]):
    print(f'CRS_{r}')
    lp = pulp.LpProblem(f"CRS_Primal{r}", pulp.LpMaximize)
    # Define Decicison Variables
    var_key = range(df.shape[0])
    U1 = pulp.LpVariable.dict("u1", var_key, lowBound=0)
    V1 = pulp.LpVariable.dict("v1", var_key, lowBound=0)

    # Define Objective Function
    lp += U1[r]*df[Y[0]][r]

    # Define Constraints
    lp += V1[r]*df[X[0]][r] == 1
    for k in var_key:
        if r != k:
            lp += U1[r]*df[Y[0]][k]  <= V1[r]*df[X[0]][k]

    # Solve Model
    try:
        s = lp.solve(GLPK_CMD(msg=False))
        sol_dict = {1: 'Optimal', 2: 'Not Solved',
                    3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
        if 1 <= s <= 5:
            print(f"({r}) Solver status:", sol_dict[s])
    except Exception as e:
        print("\n\nmodel.solve() has raised an ERROR:", e)
    OE_crs.append(pulp.value(lp.objective))
    U_crs.append(pulp.value(U1[r]))
    V_crs.append(pulp.value(V1[r]))
    print(f'U1 = {pulp.value(U1[r]):5<}, V1 = {pulp.value(V1[r]):5<}')

df['Efficiency (CRS/Primal)'] = OE_crs
df['Input Weight (CRS/Primal)'] = V_crs
df['Output Weight (CRS/Primal)'] = U_crs

CRS_0
(0) Solver status: Optimal
U1 = 0.5, V1 = 1.0
CRS_1
(1) Solver status: Optimal
U1 = 0.333333, V1 = 0.5
CRS_2
(2) Solver status: Optimal
U1 = 0.125, V1 = 0.25
CRS_3
(3) Solver status: Optimal
U1 = 0.0833333, V1 = 0.166667
CRS_4
(4) Solver status: Optimal
U1 = 0.0555556, V1 = 0.111111
CRS_5
(5) Solver status: Optimal
U1 = 0.1, V1 = 0.2
CRS_6
(6) Solver status: Optimal
U1 = 0.125, V1 = 0.25
CRS_7
(7) Solver status: Optimal
U1 = 0.05, V1 = 0.1
CRS_8
(8) Solver status: Optimal
U1 = 0.0625, V1 = 0.125


In [14]:
df

,DMU,Input,Output,Efficiency (CRS/Primal),Input Weight (CRS/Primal),Output Weight (CRS/Primal)
0,A,1,1,0.500000,1.000000,0.500000
1,B,2,4,1.333332,0.500000,0.333333
2,C,4,6,0.750000,0.250000,0.125000
3,D,6,7,0.583333,0.166667,0.083333
4,E,9,8,0.444445,0.111111,0.055556
5,F,5,3,0.300000,0.200000,0.100000
6,G,4,1,0.125000,0.250000,0.125000
7,H,10,7,0.350000,0.100000,0.050000
8,I,8,4,0.250000,0.125000,0.062500


## OE, CRS, Dual

In [15]:
OE_dual = []
Lamda_sum = []
Theta_crs = []
for r in range(df.shape[0]):
    print(f'CRS_Dual_{r}')
    lp = pulp.LpProblem(f"CRS_Dual_{r}", pulp.LpMinimize)
    # Define Decicison Variables
    var_key = range(df.shape[0])
    TH = pulp.LpVariable(name="theta_r", lowBound=0)
    LM = pulp.LpVariable.dict("lamda", var_key, lowBound=0)

    # Define Objective Function
    lp += TH

    # Define Constraints
    for x in X:
        lp += pulp.lpSum(LM[k]*df[x][k] for k in var_key) <= TH*df[x][r]
    for y in Y:
        lp += pulp.lpSum(LM[k]*df[y][k] for k in var_key) >= df[y][r]

    # Solve Model
    try:
        s = lp.solve(GLPK_CMD(msg=False))
        sol_dict = {1: 'Optimal', 2: 'Not Solved',
                    3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
        if 1 <= s <= 5:
            print(f"({r}) Solver status:", sol_dict[s])
    except Exception as e:
        print("\n\nmodel.solve() has raised an ERROR:", e)
    OE_dual.append(pulp.value(lp.objective))
    Lamda_sum.append(pulp.value(pulp.lpSum(LM[k] for k in var_key)))
    Theta_crs.append(pulp.value(TH))

df['Overall Efficiency (CRS/Dual)'] = OE_dual
df['lamda_sum (CRS/Dual)'] = Lamda_sum
df['theta (CRS/Dual)'] = Theta_crs
VRS = ['IRS' if se < 1 else ('DRS' if se > 1 else 'CRS') for se in Lamda_sum]
df['VRS'] = VRS

CRS_Dual_0
(0) Solver status: Optimal
CRS_Dual_1
(1) Solver status: Optimal
CRS_Dual_2
(2) Solver status: Optimal
CRS_Dual_3
(3) Solver status: Optimal
CRS_Dual_4
(4) Solver status: Optimal
CRS_Dual_5
(5) Solver status: Optimal
CRS_Dual_6
(6) Solver status: Optimal
CRS_Dual_7
(7) Solver status: Optimal
CRS_Dual_8
(8) Solver status: Optimal


In [16]:
df

,DMU,Input,Output,Efficiency (CRS/Primal),Input Weight (CRS/Primal),Output Weight (CRS/Primal),Overall Efficiency (CRS/Dual),lamda_sum (CRS/Dual),theta (CRS/Dual),VRS
0,A,1,1,0.500000,1.000000,0.500000,0.500000,0.25,0.500000,IRS
1,B,2,4,1.333332,0.500000,0.333333,1.000000,1.00,1.000000,CRS
2,C,4,6,0.750000,0.250000,0.125000,0.750000,1.50,0.750000,DRS
3,D,6,7,0.583333,0.166667,0.083333,0.583333,1.75,0.583333,DRS
4,E,9,8,0.444445,0.111111,0.055556,0.444444,2.00,0.444444,DRS
5,F,5,3,0.300000,0.200000,0.100000,0.300000,0.75,0.300000,IRS
6,G,4,1,0.125000,0.250000,0.125000,0.125000,0.25,0.125000,IRS
7,H,10,7,0.350000,0.100000,0.050000,0.350000,1.75,0.350000,DRS
8,I,8,4,0.250000,0.125000,0.062500,0.250000,1.00,0.250000,CRS


## TE, VRS, Dual

In [17]:
TE = []
VRS = []
for r in range(df.shape[0]):
    print(f'CRS_Duel_{r}')
    lp = pulp.LpProblem(f"CRS_Duel_{r}", pulp.LpMinimize)
    # Define Decicison Variables
    var_key = range(df.shape[0])
    TH = pulp.LpVariable(name="theta_r", lowBound=0)
    LM = pulp.LpVariable.dict("lamda", var_key, lowBound=0)

    # Define Objective Function
    lp += TH

    # Define Constraints
    for x in X:
        lp += pulp.lpSum(LM[k]*df[x][k] for k in var_key) <= TH*df[x][r]
    for y in Y:
        lp += pulp.lpSum(LM[k]*df[y][k] for k in var_key) >= df[y][r]
    lp += pulp.lpSum(LM[k] for k in var_key) == 1

    # Solve Model
    try:
        s = lp.solve(GLPK_CMD(msg=False))
        sol_dict = {1: 'Optimal', 2: 'Not Solved',
                    3: 'Infeasible', 4: 'Unbounded', 5: 'Undefined'}
        if 1 <= s <= 5:
            print(f"({r}) Solver status:", sol_dict[s])
    except Exception as e:
        print("\n\nmodel.solve() has raised an ERROR:", e)
    # if 
    TE.append(pulp.value(TH))
    # VRS.append(pulp.value(LM[r]))

df['TE (VRS/Duel)'] = TE
df['SE'] = np.array(OE_dual) / np.array(TE)

CRS_Duel_0
(0) Solver status: Optimal
CRS_Duel_1
(1) Solver status: Optimal
CRS_Duel_2
(2) Solver status: Optimal
CRS_Duel_3
(3) Solver status: Optimal
CRS_Duel_4
(4) Solver status: Optimal
CRS_Duel_5
(5) Solver status: Optimal
CRS_Duel_6
(6) Solver status: Optimal
CRS_Duel_7
(7) Solver status: Optimal
CRS_Duel_8
(8) Solver status: Optimal


In [18]:
df

,DMU,Input,Output,Efficiency (CRS/Primal),Input Weight (CRS/Primal),Output Weight (CRS/Primal),Overall Efficiency (CRS/Dual),lamda_sum (CRS/Dual),theta (CRS/Dual),VRS,TE (VRS/Duel),SE
0,A,1,1,0.500000,1.000000,0.500000,0.500000,0.25,0.500000,IRS,1.000000,0.500000
1,B,2,4,1.333332,0.500000,0.333333,1.000000,1.00,1.000000,CRS,1.000000,1.000000
2,C,4,6,0.750000,0.250000,0.125000,0.750000,1.50,0.750000,DRS,1.000000,0.750000
3,D,6,7,0.583333,0.166667,0.083333,0.583333,1.75,0.583333,DRS,1.000000,0.583333
4,E,9,8,0.444445,0.111111,0.055556,0.444444,2.00,0.444444,DRS,1.000000,0.444444
5,F,5,3,0.300000,0.200000,0.100000,0.300000,0.75,0.300000,IRS,0.333333,0.900001
6,G,4,1,0.125000,0.250000,0.125000,0.125000,0.25,0.125000,IRS,0.250000,0.500000
7,H,10,7,0.350000,0.100000,0.050000,0.350000,1.75,0.350000,DRS,0.600000,0.583333
8,I,8,4,0.250000,0.125000,0.062500,0.250000,1.00,0.250000,CRS,0.250000,1.000000
